In [ ]:
# Dereplication of Blodgett2021 and Heat 2022 viral contigs combined

cd ~/blodgett_combined

# starting input files
blodgett2021.vib.contigs.fna
    # count how many total contigs
    grep -c contig blodgett2021.vib.contigs.fna
    # 327,006 contigs 
heat2022.vib.contigs.fna
    # count how many total contigs
    grep -c contig heat2022.vib.contigs.fna
    # 141,638 contigs
    # I already ran CDHIT on this, so I'm just going to combine the output with blodgett output once I get that
    clustered_heat.fna
    # 34,948 contigs
    
# make cdhit directory
mkdir cdhit

# cluster blodgett alone using CD HIT
# rename file to use for CD Hit (input.fna)
cp blodgett2021.vib.contigs.fna ./input.fna

# create CD HIT script
nano cdhitblodgett21.sh

-----------------
#!/bin/bash
#SBATCH --job-name=cdhit
#SBATCH --nodes=1
#SBATCH -t 24:00:00
#SBATCH --ntasks=10
#SBATCH --output=cdhit%j.out
#SBATCH --error=cdhit%j.err
#SBATCH --partition=bmh

module load cdhit
cd-hit-est -i input.fna -o clustered_blodgett.fna \
-c 0.95 -aS 0.85 -M 14000 -T 10
-----------------

# run CD HIT script for blodgett21 contigs
sbatch --output=./log/blodgett21.log --error=./err/blodgett21.err cdhitblodgett21.sh

# number of contigs after CD HIT: 327,006 --> 242,971

# combine CD HIT outputs from both experiments and then run CD HIT together
cat clustered* > blodgett2122.vib.contigs.fna 


# calculate total number of contigs before final CD HIT: 
grep -c contig blodgett2122.vib.contigs.fna
# total contigs = 277,919

# rename file to use for CD Hit (input.fna)
cp blodgett2122.vib.contigs.fna ./input.fna

# create CD HIT script
nano cdhit.sh

-----------------
#!/bin/bash
#SBATCH --job-name=cdhit
#SBATCH --nodes=1
#SBATCH -t 24:00:00
#SBATCH --ntasks=10
#SBATCH --output=cdhit%j.out
#SBATCH --error=cdhit%j.err
#SBATCH --partition=bmh

module load cdhit
cd-hit-est -i input.fna -o clustered_all.fna \
-c 0.95 -aS 0.85 -M 15000 -T 10
-----------------

# run CD HIT on all contigs
sbatch --output=./log/all.log --error=./err/all.err cdhit.sh

# number of contigs after CD HIT
grep -c contig clustered_all.fna
    # 277,919 --> 272,017

# now run dereplication

# take the concatenated file and move it to the appropriate split_contig folder within a drep/"subset"/split_contigs
mv clustered_all.fna ../drep/all/split_contigs

# separate the concatenated fasta file into multiple ones
# use screen because this will take awhile...
screen
cd ~/blodgett_combined/drep/all/split_contigs
awk '/^>/ {OUT=substr($0,2) ".fa"}; OUT {print >OUT}' *.fna 
    # if using screen...you can monitor how close it is to done by counting file names
        ls -1 | wc -l        
# detach from screen: ctrl a + d
# to see screens running
screen -r
# to end screen
exit

# once done, remove the concatenated file from folder (you can move up a folder)
mv clustered_all.fna ../

# divide contigs into ~ six batches 
mkdir batch1 batch2 batch3 batch4 batch5 batch6

# within each batch, make a split_contigs folder
cd batch1
mkdir split_contigs

# because 272,017 is too many for one run of drep, it will run out of memory
# each batch needs to be less than 50,000, so divide into 6 batches
cd all/split_contigs
ls | head -n 45337 | xargs -I {} mv {} ../../batch1/split_contigs
ls | head -n 45337 | xargs -I {} mv {} ../../batch2/split_contigs
ls | head -n 45337 | xargs -I {} mv {} ../../batch3/split_contigs
ls | head -n 45337 | xargs -I {} mv {} ../../batch4/split_contigs
ls | head -n 45337 | xargs -I {} mv {} ../../batch5/split_contigs
ls | head -n 45332 | xargs -I {} mv {} ../../batch6/split_contigs


# make drep script
nano drep.sh
-----------------------------------------------------------------------------------------

#!/bin/bash
#SBATCH -D /home/seuge91/blodgett_combined/drep
#SBATCH --job-name=drep
#SBATCH --nodes=1
#SBATCH -t 72:00:00
#SBATCH --ntasks=24
#SBATCH --mem=900GB
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

# activate personal conda env
source /home/csantosm/initconda
conda activate DREP

treatment=${1}

cd /home/seuge91/blodgett_combined/drep/${treatment}

dRep dereplicate ./${treatment}_dRep \
-g ./split_contigs/*.fa \
--S_algorithm ANImf \
-sa 0.95 \
-nc 0.85 \
-l 10000 \
-N50W 0 \
-sizeW 1 \
--ignoreGenomeQuality \
--clusterAlg single \
-p 24

#getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
-----------------------------------------------------------------------------------------
# run the script giving the variable of the subset folder that has the split_contigs directory
sbatch drep.sh batch1
sbatch drep.sh batch2
sbatch drep.sh batch3
sbatch drep.sh batch4
sbatch drep.sh batch5
sbatch drep.sh batch6
-----------------------------------------------------------------------------------------

# number of contigs after drep: ?? --> ?? 
# batch1: 45337 --> 42431 
# batch2: 45337 --> 42230

# this is not enough reduction to keep running depreplication...
# so for now, I'm just moving forward with clustering from CD HIT alone

# how to download the CD HIT output file
scp seuge91@farm.cse.ucdavis.edu:/home/seuge91/blodgett_combined/cdhit/output/clustered_all.fna.clstr /mnt/c/Users/segeo/Downloads
    
# figuring out nucleotides of the renamed_contigs 
# none of these commands worked!
# the reason I'm doing this is to figure out which of the heat tolerant contigs identified in the heating study
# could be identified in blodgett prescribed burn and in what samples and at what abundance
# the problem is, since I dereplicated them all together, the contig name used in the heat study might have been clustered under
# a different name (the contig assembled from the prescribed burn study might be the one that is representing the cluster)
# So, I need to identify which contigs were renamed, but the other problem is the CD HIT output file doesn't actually have the 
# full name of the contigs that are clustered together (the contig names are too long, and so they get cut off), 
# so I can't identify it simply. The only way I can do it is by looking at 
# nucleotide number since the output file has that information, so that's what I've been trying to do, pull out the specific
# contigs from the input file and then calculate the number of nucleotides associated with each contig, but
# none of these commands are working!!!


grep -A 1 -f renamed_contigs.txt input.fna | grep -v "^--$" > selected_contigs.fasta

grep -F -f renamed_contigs.txt -A 1 input.fna | grep -v "^--$" > selected_contigs.fasta

awk 'NR==FNR{a[$0];next} /^>/{p=($1 in a)} p' renamed_contigs.txt input.fna > selected_contigs.fasta


# I'm going to rerun the final CD-HIT but change the output file so that it doesn't cut off the contig name. (add -d ?)
# Then I will need to see if the .clstr file is the exact same as the original .clstr file 

# create new CD HIT script
nano new_cdhit.sh

-----------------
#!/bin/bash
#SBATCH --job-name=new_cdhit
#SBATCH --nodes=1
#SBATCH -t 24:00:00
#SBATCH --ntasks=10
#SBATCH --output=new_cdhit%j.out
#SBATCH --error=new_cdhit%j.err
#SBATCH --partition=bmh

module load cdhit
cd-hit-est -i input.fna -o clustered_all_new.fna \
-c 0.95 -aS 0.85 -d 0 -M 15000 -T 10
-----------------

# run CD HIT on all contigs
sbatch --output=./log/new_all.log --error=./err/new_all.err new_cdhit.sh

# to see if the output files are identical
diff -q clustered_all.fna clustered_all_new.fna

# yes! Identical!


In [ ]:
# read mapping

# make the directory structure for bowtie
mkdir bowtie
cd bowtie
mkdir err log alignments ref coverm

# input will be final CD HIT file
drep/all/clustered_all.fna

# convert this to a .fa file
cp clustered_all.fna clustered_all.fa

# make bowtie reference script
nano bowtie_ref.sh

# I needed to increase memory to 200GB because such a large dataset...
----------------------------

#!/bin/bash
#SBATCH --job-name=bt2ref
#SBATCH --nodes=1
#SBATCH -t 12:00:00
#SBATCH --ntasks=1
#SBATCH --mem=200GB
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

module load bowtie2

cd /home/seuge91/blodgett_combined/bowtie/ref/

bowtie2-build ../clustered_all.fa all_vibrant_cdhit

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------

# run the script to index the reference database
sbatch --output=../bowtie/log/ref.log --error=../bowtie/err/ref.err bowtie_ref.sh

# map the reads against the database
# make mapping script
nano bowtie_map.sh

----------------------------

#!/bin/bash
#SBATCH --job-name=bt2map
#SBATCH --nodes=1
#SBATCH -t 6:00:00
#SBATCH --ntasks=48
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

#load modules
module load bowtie2
module load samtools

path=/home/seuge91/blodgett_combined/
sample=${1}

cd /home/seuge91/blodgett_combined/bowtie/ref/

bowtie2 -x all_vibrant_cdhit -p 48 \
-1 ${path}reads/rmphix/${sample}_R1_rmphix.fq.gz \
-2 ${path}reads/rmphix/${sample}_R2_rmphix.fq.gz \
-S ${path}bowtie/alignments/${sample}.vib.sam \
--sensitive

cd /home/seuge91/blodgett_combined/bowtie/alignments
samtools view -F 4 -bS ${sample}.vib.sam | samtools sort > ${sample}.vib.sI.bam
samtools index ${sample}.vib.sI.bam

rm ${sample}.vib.sam

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------

# edit file names first:
for file in *_rmphix_combined.fq.gz; do mv "$file" "${file/_combined/}"; done

# run mapping script
for sample in $(<../all_sample.txt)
do
  sbatch --output=../bowtie/log/${sample}.map.log --error=../bowtie/err/${sample}.map.err bowtie_map.sh $sample
done


In [ ]:
# generate vOTU table

# make coverM script
nano coverm.sh

----------------------------

#!/bin/bash
#SBATCH --job-name=coverm
#SBATCH --nodes=1
#SBATCH -t 10:00:00
#SBATCH --ntasks=1
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

## you need to run the command from my coverm folder
cd /home/csantosm/software/coverm-x86_64-unknown-linux-musl-0.6.1/

path=/home/seuge91/blodgett_combined/bowtie/

./coverm contig -m trimmed_mean --min-covered-fraction 0.75 -b ${path}/alignments/*.bam > ${path}/coverm/all.good.75.tmean.tsv

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------

# run script
sbatch --output=../bowtie/log/coverm.log --output=../bowtie/err/coverm.err coverm.sh

# download table

scp seuge91@farm.cse.ucdavis.edu:/home/seuge91/blodgett_combined/bowtie/coverm/*.tsv /mnt/c/Users/segeo/Downloads

In [ ]:
# Generating Count Table for Differential Abundance

# make covermcount script
nano covermcount.sh

----------------------------

#!/bin/bash
#SBATCH --job-name=covermcount
#SBATCH --nodes=1
#SBATCH -t 10:00:00
#SBATCH --ntasks=1
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

## you need to run the command from my coverm folder
cd /home/csantosm/software/coverm-x86_64-unknown-linux-musl-0.6.1/

path=/home/seuge91/blodgett_combined/bowtie/

./coverm contig -m count -b ${path}/alignments/*.bam > ${path}/coverm/all.count.tsv

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
        
----------------------------
    
# Download table
    
scp seuge91@farm.cse.ucdavis.edu:/home/seuge91/heat/bowtie/coverm/*.count.tsv /mnt/c/Users/segeo/Downloads

In [ ]:
# host prediction

mkdir iphop
mkdir err log split_db results
nano iphop_split.sh

----------------------------

#!/bin/bash
#SBATCH -D /home/seuge91/blodgett_combined/iphop/
#SBATCH --job-name=iphop_split
#SBATCH --nodes=1
#SBATCH -t 1:00:00
#SBATCH --ntasks=1
#SBATCH --partition=bmm

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

source /home/csantosm/initconda
conda activate IPHOP

set=${1}

full_db=/home/seuge91/blodgett_combined/bowtie/clustered_all.fa
split_db=/home/seuge91/blodgett_combined/iphop/split_db

iphop split --input_file ${full_db} --split_dir ${split_db}

#getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed

----------------------------

sbatch --output=./log/iphop_split.log --error=./err/iphop_split.err iphop_split.sh

nano iphop_predict.sh

----------------------------

#!/bin/bash
#SBATCH -D /home/seuge91/blodgett_combined/iphop/
#SBATCH --job-name=iphop_predict
#SBATCH --nodes=1
#SBATCH -t 24:00:00
#SBATCH --mem=128GB
#SBATCH --ntasks=24
#SBATCH --partition=bmm

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

source /home/csantosm/initconda
conda activate IPHOP

batch=${1}

iphop_db=/home/csantosm/databases/IPHOP_db/Sept_2021_pub

cd /home/seuge91/blodgett_combined/iphop

iphop predict --fa_file ./split_db/${batch}.fna --out_dir ./results/${batch} --db_dir ${iphop_db} --num_threads 24

#getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------


batches="batch_00000 batch_00001 batch_00002 batch_00003 batch_00004 batch_00005 batch_00006 batch_00007 batch_00008 batch_00009 batch_00010 batch_00011 batch_00012 batch_00013 batch_00014 batch_00015 batch_00016 batch_00017 batch_00018 batch_00019 batch_00020 batch_00021 batch_00022 batch_00023 batch_00024 batch_00025 batch_00026 batch_00027 batch_00028 batch_00029 batch_00030 batch_00031 batch_00032 batch_00033 batch_00034 batch_00035 batch_00036 batch_00037 batch_00038 batch_00039 batch_00040 batch_00041 batch_00042 batch_00043 batch_00044 batch_00045 batch_00046 batch_00047 batch_00048 batch_00049 batch_00050 batch_00051 batch_00052 batch_00053 batch_00054 batch_00055 batch_00056 batch_00057 batch_00058 batch_00059 batch_00060 batch_00061 batch_00062 batch_00063 batch_00064 batch_00065 batch_00066 batch_00067 batch_00068 batch_00069 batch_00070 batch_00071 batch_00072 batch_00073 batch_00074 batch_00075 batch_00076 batch_00077 batch_00078 batch_00079 batch_00080 batch_00081 batch_00082 batch_00083 batch_00084 batch_00085 batch_00086 batch_00087 batch_00088 batch_00089 batch_00090 batch_00091 batch_00092 batch_00093 batch_00094 batch_00095 batch_00096 batch_00097 batch_00098 batch_00099 batch_00100 batch_00101 batch_00102 batch_00103 batch_00104 batch_00105 batch_00106 batch_00107 batch_00108 batch_00109 batch_00110 batch_00111 batch_00112 batch_00113 batch_00114 batch_00115 batch_00116 batch_00117 batch_00118 batch_00119 batch_00120 batch_00121 batch_00122 batch_00123 batch_00124 batch_00125 batch_00126 batch_00127 batch_00128 batch_00129 batch_00130 batch_00131 batch_00132 batch_00133 batch_00134 batch_00135 batch_00136 batch_00137 batch_00138 batch_00139 batch_00140 batch_00141 batch_00142 batch_00143 batch_00144 batch_00145 batch_00146 batch_00147 batch_00148 batch_00149 batch_00150 batch_00151 batch_00152 batch_00153 batch_00154 batch_00155 batch_00156 batch_00157 batch_00158 batch_00159 batch_00160 batch_00161 batch_00162 batch_00163 batch_00164 batch_00165 batch_00166 batch_00167 batch_00168 batch_00169 batch_00170 batch_00171 batch_00172 batch_00173 batch_00174 batch_00175 batch_00176 batch_00177 batch_00178 batch_00179 batch_00180 batch_00181 batch_00182 batch_00183 batch_00184 batch_00185 batch_00186 batch_00187 batch_00188 batch_00189 batch_00190 batch_00191 batch_00192 batch_00193 batch_00194 batch_00195 batch_00196 batch_00197 batch_00198 batch_00199 batch_00200 batch_00201 batch_00202 batch_00203 batch_00204 batch_00205 batch_00206 batch_00207 batch_00208 batch_00209 batch_00210 batch_00211 batch_00212 batch_00213 batch_00214 batch_00215 batch_00216 batch_00217 batch_00218 batch_00219 batch_00220 batch_00221 batch_00222 batch_00223 batch_00224 batch_00225 batch_00226 batch_00227 batch_00228 batch_00229 batch_00230 batch_00231 batch_00232 batch_00233 batch_00234 batch_00235 batch_00236 batch_00237 batch_00238 batch_00239 batch_00240 batch_00241 batch_00242 batch_00243 batch_00244 batch_00245 batch_00246 batch_00247 batch_00248 batch_00249 batch_00250 batch_00251 batch_00252 batch_00253 batch_00254 batch_00255 batch_00256 batch_00257 batch_00258 batch_00259 batch_00260 batch_00261 batch_00262 batch_00263 batch_00264 batch_00265 batch_00266 batch_00267 batch_00268 batch_00269 batch_00270 batch_00271 batch_00272"
for batch in $batches
do
sbatch --output=./log/${batch}.log --error=./err/${batch}.err iphop_predict.sh $batch
done

batches="batch_00087 batch_00077 batch_00073"
for batch in $batches
do
sbatch --output=./log/${batch}.log --error=./err/${batch}.err iphop_predict.sh $batch
done

# combine results from batches
# get header from one of the batches
cd results
cd batch_00000
head -n 1 Host_prediction_to_genome_m90.csv > all_Host_prediction_to_genome_m90.csv
head -n 1 Host_prediction_to_genus_m90.csv > all_Host_prediction_to_genus_m90.csv
head -n 10 Detailed_output_by_tool.csv > all_Detailed_output_by_tool.csv
# move files up a folder
mv all_Host_prediction_to_genome_m90.csv ..
mv all_Host_prediction_to_genus_m90.csv ..
mv all_Detailed_output_by_tool.csv ..
# get body of each batch and concatenate to main file
cd ../
tail -n +2 -q batch_*/Host_prediction_to_genome_m90.csv >> all_Host_prediction_to_genome_m90.csv
tail -n +2 -q batch_*/Host_prediction_to_genus_m90.csv >> all_Host_prediction_to_genus_m90.csv
tail -n +11 batch_*/Detailed_output_by_tool.csv >> all_Detailed_output_by_tool.csv

# download files
scp seuge91@farm.cse.ucdavis.edu:/home/seuge91/blodgett_combined/iphop/results/*.csv /mnt/c/Users/segeo/Downloads